## Final Project Submission

Please fill out:
* Student name: clyde ochieng
* Student pace: full time
* Scheduled project review date/time: 
* Instructor name: Nikita  
* Blog post URL:


# Kings County Housing Analysis with Multiple Linear Regression

## Overview


In the grand scheme of things, a youthful duo is in the midst of orchestrating the sale of their abode, harboring aspirations to elevate its market worth with minimal financial resources allocated for refurbishments. In a strategic move, the couple has opted to employ Multiple Linear Regression Modeling to meticulously scrutinize and prognosticate house sales trends in King County, leveraging specific attributes or variables. This analytical approach is poised to empower them with insights to navigate the market intelligently, thereby facilitating judicious decisions for maximizing profitability.

Following a rigorous evaluation process and numerous iterations of our linear regression models, it has been ascertained that the dimensions of living space and the quality grade of the construction exhibit the most pronounced correlation with an augmented selling price of the property.

## Business Problem


The business quandary at hand revolves around providing insightful counsel to prospective homeowners regarding the potential impact of home renovations on the estimated value of their properties, alongside quantifying the magnitude of such enhancements.

In addressing this issue, we aim to equip homeowners, both buyers and sellers, with invaluable insights that can inform their decisions in the real estate market. Our focus lies in scrutinizing key factors such as building grade and square footage of living space, among others, to unveil the correlations that exist between these features and home sale prices. This comprehensive analysis seeks to elucidate the most influential attributes driving property values, thereby empowering stakeholders to make informed and strategic decisions pertaining to their real estate endeavors.

### Hypothesis


In crafting our hypotheses, we put forward the following propositions:

Null Hypothesis (H0): Within our model, we posit that there is an absence of any substantial association between the independent variables and the dependent variable (target).

Alternative Hypothesis (H1): Contrarily, we assert that a meaningful relationship does exist between the independent variables and the dependent variable (target) within our model.

To guide our analytical journey and ensure robust conclusions, we have established a significance level (alpha) of 0.05. This chosen threshold will serve as a pivotal benchmark against which we evaluate the statistical significance of our findings and shape our ultimate recommendations for stakeholders.

### Questions to be analyzed


Let's articulate the questions we aim to explore through our analysis:

Q1: Which features show the strongest correlation with the price of a home?

Q2: Among the predicting variables, which features exhibit the most significant correlations with each other?

Q3: What combinations of features offer the most reliable predictions for home prices?

These questions form the cornerstone of our investigation, guiding our efforts to uncover key insights that will assist homeowners, buyers, and sellers alike in navigating the complex landscape of real estate with confidence and clarity.

## Data Understanding


In our quest to gain a comprehensive understanding of the data under scrutiny, we turn our attention to the King County Housing dataset. This dataset encapsulates a wealth of information pertaining to over 21,000 homes situated in King County.

Each entry in the dataset provides detailed insights into various attributes of the homes, encompassing factors such as the number of bedrooms, bathrooms, and floors, as well as the square footage of both living space and lot, the zip code, building grade, and condition, among others.

The dataset comprises a multitude of features, each potentially contributing to the final sale price of homes in King County. Below, we delineate the descriptions of these features, laying the groundwork for a thorough exploration and analysis.

##### King County Housing Data Columns 

Below is a comprehensive breakdown of the columns present in the King County Housing dataset:

- `id`: A unique identifier assigned to each house.
- `date`: The date on which the house was sold. (This column will be ignored for our analysis.)
- `price`: The sale price of the house, serving as our prediction target.
- `bedrooms`: The number of bedrooms in the house.
- `bathrooms`: The number of bathrooms in the house.
- `sqft_living`: The square footage of the living space within the house.
- `sqft_lot`: The square footage of the lot on which the house is situated.
- `floors`: The number of floors or levels present in the house.
- `waterfront`: A binary indicator denoting whether the house is situated on a waterfront.
- `view`: An indication of the quality of the view from the house. (This column will be ignored for our analysis.)
- `condition`: An assessment of the overall condition of the house.
- `grade`: An overall grade assigned to the house.
- `sqft_above`: The square footage of the house, excluding the basement. (This column will be ignored for our analysis.)
- `sqft_basement`: The square footage of the basement within the house. (This column will be ignored for our analysis.)
- `yr_built`: The year in which the house was built.
- `yr_renovated`: The year in which the house was renovated. (This column will be ignored for our analysis.)
- `zipcode`: The ZIP code corresponding to the location of the house. (This column will be ignored for our analysis.)
- `lat`: The latitude coordinate of the house. (This column will be ignored for our analysis.)
- `long`: The longitude coordinate of the house. (This column will be ignored for our analysis.)
- `sqft_living15`: The square footage of interior housing living space for the nearest 15 neighbors. (This column will be ignored for our analysis.)
- `sqft_lot15`: The square footage of the land lots of the nearest 15 neighbors. (This column will be ignored for our analysis.)


### Explotory Data Analyis

As we set sail on our journey of exploration, our first port of call involves the pivotal task of importing the dataset. This foundational step lays the groundwork for our subsequent analyses and insights, ushering us into the realm of data-driven discovery and understanding.

In [1]:
#load necessary modules 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as stats
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.formula.api import ols
from sklearn import datasets, linear_model
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

##### Eliminating Irrelevant Columns
In our quest for clarity and focus, we embark on the process of streamlining our dataset by removing columns that are deemed irrelevant to our current analysis. This strategic pruning ensures that we hone in on the most pertinent features, thereby enhancing the efficacy and interpretability of our subsequent explorations.

In [2]:
# reading the csv file
# eliminating irrelevant columns
df = pd.read_csv('data/kc_house_data.csv').drop(['date',
                                                 'view', 
                                                 'sqft_above', 
                                                 'sqft_basement', 
                                                 'yr_renovated',
                                                 'zipcode', 
                                                 'lat', 
                                                 'long', 
                                                 'sqft_living15',
                                                 'sqft_lot15'], axis = 1)
# previewing the DataFrame
df.head()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,grade,yr_built
0,7129300520,221900.0,3,1.00,1180,5650,1.0,NaN,Average,7 Average,1955
1,6414100192,538000.0,3,2.25,2570,7242,2.0,NO,Average,7 Average,1951
2,5631500400,180000.0,2,1.00,770,10000,1.0,NO,Average,6 Low Average,1933
3,2487200875,604000.0,4,3.00,1960,5000,1.0,NO,Very Good,7 Average,1965
4,1954400510,510000.0,3,2.00,1680,8080,1.0,NO,Average,8 Good,1987


In [3]:
df.shape

(21597, 11)

The dataset comprises 21,597 houses, each characterized by 11 features.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           21597 non-null  int64  
 1   price        21597 non-null  float64
 2   bedrooms     21597 non-null  int64  
 3   bathrooms    21597 non-null  float64
 4   sqft_living  21597 non-null  int64  
 5   sqft_lot     21597 non-null  int64  
 6   floors       21597 non-null  float64
 7   waterfront   19221 non-null  object 
 8   condition    21597 non-null  object 
 9   grade        21597 non-null  object 
 10  yr_built     21597 non-null  int64  
dtypes: float64(3), int64(5), object(3)
memory usage: 1.8+ MB


In [5]:
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,yr_built
count,2.159700e+04,2.159700e+04,21597.000000,21597.000000,21597.000000,2.159700e+04,21597.000000,21597.000000
mean,4.580474e+09,5.402966e+05,3.373200,2.115826,2080.321850,1.509941e+04,1.494096,1970.999676
std,2.876736e+09,3.673681e+05,0.926299,0.768984,918.106125,4.141264e+04,0.539683,29.375234
min,1.000102e+06,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,1900.000000
25%,2.123049e+09,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,1951.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,1975.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068500e+04,2.000000,1997.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,2015.000000


In [6]:
# indepth statistics for price dataframe.
df['price'].describe()

count    2.159700e+04
mean     5.402966e+05
std      3.673681e+05
min      7.800000e+04
25%      3.220000e+05
50%      4.500000e+05
75%      6.450000e+05
max      7.700000e+06
Name: price, dtype: float64


The dataset indicates that the average price of homes is $540,297. Prices within the dataset range from $78,000 to $8,000,000. Additionally, the median house price is reported to be $450,000.

In [7]:
# indepth statistics for square footage
df['sqft_living'].describe()

count    21597.000000
mean      2080.321850
std        918.106125
min        370.000000
25%       1430.000000
50%       1910.000000
75%       2550.000000
max      13540.000000
Name: sqft_living, dtype: float64

The dataset reveals that the mean square footage of living space is 2,080 sq-ft, with living space ranging from 370 sq-ft to 13,540 sq-ft. The median square footage is reported to be 1,910 sq-ft.

In [8]:
df['bedrooms'].value_counts()

bedrooms
3     9824
4     6882
2     2760
5     1601
6      272
1      196
7       38
8       13
9        6
10       3
11       1
33       1
Name: count, dtype: int64